In [1]:
import re
import json
import nltk
import numpy as np
import pandas as pd
import string
import os
import random
import codecs
from collections import OrderedDict
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer 
from sklearn.model_selection import train_test_split

allbeers = pd.read_csv('../beers/beerlist.csv')
print('number of beers = '+(str(len(allbeers))))

number of beers = 4813


In [2]:
def pre_process(text):
    # lowercase
    text=text.lower()
    #remove tags
    text=re.sub("<!--?.*?-->","",text)
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    return text

allbeers['words'] = allbeers['words'].apply(lambda x:pre_process(x))
for i in range(len(allbeers)):
    allbeers['words'][i] = allbeers['words'][i].split()
    try:
        allbeers['words'][i] = random.sample(allbeers['words'][i], 2000)
    except ValueError:
        allbeers['words'][i] = random.sample(allbeers['words'][i], len(allbeers['words'][i]))
    allbeers['words'][i] = ' '.join(allbeers['words'][i])

/home/steven/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/steven/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/steven/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/steven/anaconda2/lib/python3.7/site-packages/i

In [12]:
import pickle
# Function to remove stop words from sentences & lemmatize words.
def clean(beer):
    stop_free = " ".join([i for i in beer.lower().split() if i not in stop])
    normalized = " ".join(lemma.lemmatize(word,'v') for word in stop_free.split())
    x = normalized.split()
    y = [s for s in x if len(s) > 2]
    return y

In [37]:
beer_words = allbeers['words']
#beers = open("docs_beers.pkl",'wb')
#pickle.dump(beer_words, beers)
 
# Use 3500 beers for training.
#beer_train, beer_test = train_test_split(beer_words, test_size=0.20)
beer_train = allbeers['words'][:3500]

# Cleaning all the train beers
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
beer_clean = [clean(beer) for beer in beer_train]

In [38]:
from gensim import corpora
# Creating term dictionary of corpus, where each unique term is assigned an index.
dictionary = corpora.Dictionary(beer_clean)

# Filter terms which occurs in less than 10 beers & more than 90% of the comments
dictionary.filter_extremes(no_below=10, no_above=0.9)
 
# List of few words which are removed from dictionary as they are content neutral
stoplist = set('bad alcohol strong abv better good nothing beer beers also use make people know many call include part find become like mean often different usually take wikt come give well get since type list say change see refer actually iii aisne kinds pas ask would way something need things want every str'.split())
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
dictionary.filter_tokens(stop_ids)

In [18]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
beer_term_matrix = [dictionary.doc2bow(beer) for beer in beer_clean]

In [7]:
from gensim.models import LdaMulticore as Lda
# Creating the object for LDA model using gensim library & Training LDA model on the document term matrix.
ldamodel = Lda(beer_term_matrix, workers=3, num_topics=14, id2word = dictionary, passes=10, iterations=200)

# dump LDA model using cPickle for future use
ldafile = open('lda_model_beers_14topics.pkl','wb')
pickle.dump(ldamodel,ldafile)
ldafile.close()
 
# Print all the 50 topics
for i,topic in enumerate(ldamodel.print_topics(num_topics=14, num_words=10)):
   words = topic[1].split("+")
   print(words,"\n")

ValueError: cannot compute LDA over an empty collection (no terms)

In [59]:
def cluster_similar_documents(corpus, dirname):
    clean_beers = [clean(beer) for beer in corpus]
    test_term = [ldamodel.id2word.doc2bow(beer) for beer in clean_beers]
    beer_topics = ldamodel.get_document_topics(test_term, minimum_probability=0.20)
    for k,topics in enumerate(beer_topics):
        if topics:
            topics.sort(key = itemgetter(1), reverse=True)
            dir_name = dirname + "/" + str(topics[0][0])
            file_name = dir_name + "/" + str(k) + ".txt"
            if not os.path.exists(dir_name):
                os.makedirs(dir_name)
            fp = open(file_name,"w")
            fp.write(beers_test[k] + "\n\n" + str(topics[0][1]) )
            fp.close()
        else:
            if not os.path.exists(dirname + "/unknown"):
                os.makedirs(dirname + "/unknown")
            file_name = dirname + "/unknown/" + str(k) + ".txt"
            fp = open(file_name,"w")
            fp.write(beers_test[k])
'''
def get_related_documents(term, top, corpus):
    clean_beers = [clean(beer) for beer in corpus]
    related_beerid = []
    test_term = [ldamodel.id2word.doc2bow(beer) for beer in clean_beers]
    beer_topics = ldamodel.get_document_topics(test_term, minimum_probability=0.30)
    term_topics =  ldamodel.get_term_topics(term, minimum_probability=0.01)
    for k,topics in enumerate(beer_topics):
        if topics:
            topics.sort(key = itemgetter(1), reverse=True)
            if topics[0][0] == term_topics[0][0]:
                related_beerid.append((k, topics[0][1]))
    beerids = []
    related_beerid.sort(key = itemgetter(1), reverse=True)
    for j,beer_id in enumerate(related_beerid):
        print(beer_id[1],"\n\n",beer_test[beer_id[0]][:10])
        if j == (top-1):
            break
        beerids.append(beer_id[0])
        return beerids #doc_id[0]


def get_related_documents(term, top, corpus):
    clean_docs = [clean(doc) for doc in corpus]
    related_docid = []
    test_term = [ldamodel.id2word.doc2bow(doc) for doc in clean_docs]
    doc_topics = ldamodel.get_document_topics(test_term, minimum_probability=0.30)
    term_topics =  ldamodel.get_term_topics(term, minimum_probability=0.01)
    for k,topics in enumerate(doc_topics):
        if topics:
            topics.sort(key = itemgetter(1), reverse=True)
            if topics[0][0] == term_topics[0][0]:
                related_docid.append((k,topics[0][1]))
 
    related_docid.sort(key = itemgetter(1), reverse=True)
    for j,doc_id in enumerate(related_docid):
        print(doc_id[1],"\n\n",docs_test[doc_id[0]])
        if j == (top-1):
            break

def get_related_documents(term, top, corpus):
    clean_beers = [clean(beer) for beer in corpus]
    related_beerid = []
    test_term = [ldamodel.id2word.doc2bow(beer) for beer in clean_beers]
    beer_topics = ldamodel.get_document_topics(test_term, minimum_probability=0.30)
    term_topics =  ldamodel.get_term_topics(term, minimum_probability=0.000001)
    for k,topics in enumerate(beer_topics):
        if topics:
            topics.sort(key = itemgetter(1), reverse=True)
            if topics[0][0] == term_topics[0][0]:
                related_beerid.append((k,topics[0][1]))
    beerids = []
    related_beerid.sort(key = itemgetter(1), reverse=True)
    for j, beer_id in enumerate(related_beerid):
        print(beer_id[1])#,"\n\n",corpus[beer_id[0]])
        if j == (top-1):
              break
        beerids.append(beer_id[0])
        return beerids
'''

def get_related_documents(term, top, corpus):
    clean_docs = [clean(doc) for doc in corpus]
    related_docid = []
    test_term = [ldamodel.id2word.doc2bow(doc) for doc in clean_docs]
    doc_topics = ldamodel.get_document_topics(test_term, minimum_probability=0.30)
    term_topics =  ldamodel.get_term_topics(term, minimum_probability=0.01)
    for k,topics in enumerate(doc_topics):
        if topics:
            topics.sort(key = itemgetter(1), reverse=True)
            if topics[0][0] == term_topics[0][0]:
                related_docid.append((k,topics[0][1]))
    beerids = []
    related_docid.sort(key = itemgetter(1), reverse=True)
    for j,doc_id in enumerate(related_docid):
        print(doc_id[1],"\n\n",docs_test[doc_id[0]])
        if j == (top-1):
            break
        beerids.append(doc_id[0])
    return beerids#doc_id[0]

In [46]:
docs_fp = open("docs_beers.pkl", 'rb')
docs_all = pickle.load(docs_fp)
docs_test = docs_all#[3500:].reset_index(drop=True)

In [ ]:
a = get_related_documents("stout", 10 , docs_test)

In [61]:
a

[152, 33, 1230, 1211, 577, 827, 332, 326, 887]

In [67]:
#beers = pd.read_csv('../beers/beerlist.csv')
#beers['name'][a]
for i in a:
    print(beers['name'][i])

Hoppin' Bubbly
Powder Hound Winter Ale
HUB Lager
Milwaukee's Best Light
Invasion Pale Ale
Titanic Stout
Ayinger Jahrhundert Bier
Immort Ale
Viking Fraoch


In [9]:
import pickle
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from operator import itemgetter
import os
 
# initialize WordNetLemmatizer and get the list of english stop words
stop = set(stopwords.words('english'))
lemma = WordNetLemmatizer()
 
# Load trained LDA model
lda_fp = open("lda_model_beers_14topics.pkl", 'rb')
ldamodel = pickle.load(lda_fp)
 
# Load the beers corpus to choose 500 lines for test purpose
beers_fp = open("docs_beers.pkl", 'rb')
#beers_all = pickle.load(docs_fp)
#_test = docs_all[3500:].reset_index(drop=True)

In [10]:
beers = 
beers_all = pickle.load(beers_fp)
beer_test = beers_all[3500:].reset_index(drop=True)

In [28]:
# Get 'top' related documents given a word(term)
a = get_related_documents("chocolate", 10 , beer_test)
# performs document clustering given a set of documents
#cluster_similar_documents(docs_test,"root")

0.99922216
